# Hatch

The `hatch` is a project management tool in python. In this page we will consider the side of using `hatch` that determines the building of the package.

## First use

Here is as an example showen of the minimal setup for building a package with wheel.

You need to create a `pyproject.toml` file where the `[build-system]` table specifies `hatchling.build` as the `build-backend` and includes `hatchling` as a requirement. It must also have a `[project]` table, where `name` and `version` are specified.

---

The following file creates corresponing `pyproject.toml`.

In [6]:
mkdir -p /tmp/hatch
cd /tmp/hatch

cat << EOF > pyproject.toml
[build-system]
requires = ["hatchling"]
build-backend = "hatchling.build"

[project]
name = "hatch_test"
version = "0.0.0"
EOF

There are some default files that `hatch` will use for building packages. Check the rules [here](https://hatch.pypa.io/latest/plugins/builder/wheel/#default-file-selection).

In [3]:
cat << EOF > hatch_test.py
print("Hello, World!")
EOF

Build the package with the `hatch build` command. 

In [4]:
hatch build

──────────────────────────────────── sdist ─────────────────────────────────────
dist/hatch_test-0.0.0.tar.gz
──────────────────────────────────── wheel ─────────────────────────────────────
dist/hatch_test-0.0.0-py2.py3-none-any.whl


As a result, there will be building artifacts in the `dist` folder.

In [5]:
tree

.
├── dist
│   ├── hatch_test-0.0.0-py2.py3-none-any.whl
│   └── hatch_test-0.0.0.tar.gz
├── hatch_test.py
└── pyproject.toml

2 directories, 4 files


## Build hooks

Build hooks define code that is executed at various stages of the build. 

Check [official description](https://hatch.pypa.io/1.1/config/build/#build-hooks) on what build hook is and on [building hooks plugins](https://hatch.pypa.io/1.1/plugins/build-hook/#hatchling.builders.hooks.plugin.interface.BuildHookInterface).

---

The simpliest set up is to add the `[tool.hatch.build.hooks.custom]` table to your `pyproject.toml` and define hooks in the `hatch_build.py`. The following cells define this type of project. Where hook just prints a specific string to stdout. 

In [ ]:
mkdir -p /tmp/hook
cd /tmp/hook

cat << EOF > pyproject.toml
[build-system]
requires = ["hatchling"]
build-backend = "hatchling.build"

[project]
name = "hatch_test"
version = "0.0.0"

[tool.hatch.build.hooks.custom]
EOF

cat << EOF > hatch_build.py
from hatchling.builders.hooks.plugin.interface import BuildHookInterface

class CustomHook(BuildHookInterface):

    def initialize(self, version, build_data):
        print("Message from the custom build hook")
EOF

touch hatch_test.py

The next cell represents the building process.

In [14]:
hatch build

──────────────────────────────────── sdist ─────────────────────────────────────
Message from the custom build hook
dist/hatch_test-0.0.0.tar.gz
──────────────────────────────────── wheel ─────────────────────────────────────
Message from the custom build hook
dist/hatch_test-0.0.0-py2.py3-none-any.whl


Before each build, there is a `Message from custom build hook` - which proves that the hook was executed.

## Shared data

You can push data from the build environment to the install package using the `[tool.hatch.build.wheel.shared-data]` table. There you can define pairs `<file in build env> = <file in result env>`. It's perfect for pushing some configuration files.

---

The following cell defines the project that will throw the `my_data` file to the root of the virtual environment.

In [20]:
mkdir -p /tmp/shared_data
cd /tmp/shared_data

cat << EOF > pyproject.toml
[build-system]
requires = ["hatchling"]
build-backend = "hatchling.build"

[project]
name = "hatch_test"
version = "0.0.0"

[tool.hatch.build.targets.wheel.shared-data]
"data" = "my_data"
EOF

echo "message to package" > data

touch hatch_test.py

hatch build &> /dev/null

The following cell creates the new environment and installes the just created package there.

In [37]:
python3 -m venv venv
venv/bin/pip install dist/hatch_test-0.0.0-py2.py3-none-any.whl

Processing ./dist/hatch_test-0.0.0-py2.py3-none-any.whl

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


Now check that file has actually been moved to the `venv` folder.

In [ ]:
ls venv

bin  include  lib  lib64  my_data  pyvenv.cfg


And there is a corresponding `my_data` file in the `venv` folder, the following cell shows that it content corresponds to the content in the building environment.

In [40]:
cat venv/my_data

message to package
